In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import shapely
%matplotlib inline

# What is a steet name?
Now that we have streets sorted, we need to work out what part is the *name*. The most common full street name may be "Small Street", but I now want to separate the name, *Small*, from its desriptor *Street*.

In [3]:
streets = pd.read_csv("clean_streets_reduced.csv")
streets['name'].head()

0                    1
1          10th avenue
2          12th avenue
3    12th avenue north
4          12th street
Name: name, dtype: object

## Attempt 1
Lets keep it simple to start of with: the first part of the street name is the *name* an 

In [6]:
def get_name1(full_name):
    '''Gets the name of the street: full name minus the last word. If the street name is one word it returns
    the street name'''
    
    split_name = full_name.split()
    return split_name[0]

streets["street_name"] = streets["name"].apply(get_name1)
streets.groupby("street_name")["id"].count().sort_values(ascending=False).head()

street_name
the        247
st         108
park        95
william     73
john        70
Name: id, dtype: int64

And the most common name is "the". As in "*The* Avenue", "*The* Grand Parade" or "*The* Serpentine". In my mind "The" is an integral part of the street name. The whole name should be *The Avenue*, *The Grand Parade* or *The Serpentine*. 

The second most common is "st" as in "St. Mark Close" or "St. Thomas Road". In these cases I think the names should be "St. Mark" or "St. Thomas". 

Let's revise that funciton. If the name starts with "The" the full name is its name. Otherwise it will return everything other than the last word in the name:


In [9]:
def get_name2(full_name):
    '''Gets the name of the street: full name minus the last word. If the street name is one word it returns
    the street name'''
    
    split_name = full_name.split()
    if split_name[0]=='the':
        return full_name
    else:
        return ' '.join(split_name[:-1])

streets["street_name"] = streets["name"].apply(get_name2)
streets.groupby("street_name")["id"].count().sort_values(ascending=False).head()

street_name
park        92
            69
victoria    60
railway     56
william     54
Name: id, dtype: int64

"Park" is now the most common - which sounds about right. But <blank> is the second most commmon. These are streets with only a single word like "Broardway" or "Kingsway". Lets do another revision: now if the full name is only one word, return that word:

In [14]:
def get_name3(full_name):
    '''Gets the name of the street: full name minus the last word. If the street name is one word it returns
    the street name'''
    
    split_name = full_name.split()
    if len(split_name)==1:
        return full_name
    if split_name[0]=='the':
        return full_name
    else:
        return ' '.join(split_name[:-1])

streets["street_name"] = streets["name"].apply(get_name3)
streets.groupby("street_name")["id"].count().sort_values(ascending=False).head()


street_name
park        92
victoria    60
railway     56
william     54
short       53
Name: id, dtype: int64

This looks fine! Lets look at what the descriptor would look like. For this, I'm just taking what is left after the name:

In [24]:
def get_description(full_name):
    '''Gets the name of the street: full name minus the last word. If the street name is one word it returns
    the street name'''
    
    split_name = full_name.split()
    if len(split_name)==1:
        return None
    if split_name[0]=='the':
        return None
    else:
        return split_name[-1]

streets["street_description"] = streets["name"].apply(get_description)
streets.groupby("street_description")["id"].count().sort_values(ascending=False).head(20)


street_description
street       12755
place         6359
avenue        5862
road          4345
lane          2228
close         1732
crescent      1626
drive          969
way            841
court          672
parade         461
circuit        389
grove          286
terrace        103
offramp         97
onramp          80
glen            57
boulevard       55
north           55
south           50
Name: id, dtype: int64

In [20]:
streets["street_description"].unique()

array([None, 'avenue', 'north', 'street', 'west', 'way', 'road',
       'circuit', '2140', 'place', 'close', 'lane', 'row', 'drive',
       'parade', 'exit', 'crescent', 'grove', 'court', 'terace',
       'reserve', 'south', 'east', 'gardens', 'st', 'off', 'onramp',
       'offramp', 'promenade', 'rise', 'square', 'atreet', 'ave', 'mews',
       'glen', 'terrace', 'closed)', 'circle', 'boulevarde', 'boulevard',
       'plave', 'grovet', 'africa', 'asia', 'europe', 'oceania',
       'americas', 'green', 'garden', 'bypass', 'plaza', 'island', 'link',
       'view', 'prade', 'cvenue', 'esplanade', 'cove', 'bridge',
       'parkway', 'promatary', 'serviceway', 'ramp', 'club', 'ridge',
       'walk', 'trail', 'highway', 'southbound', 'end', 'common', 'bend',
       'loop', 'parking', 'entry', 'path', 'deviation', 'area', 'only',
       'station', 'stops', 'expressway', 'chase', 'park', 'avenuue',
       'layover)', 'dtreet', 'glade', 'raiver', 'toe', 'tunnel', 'rd',
       '6-12', 'cycleway

This mostly looks ok, but then we see descriptions like 'north' and 'south' which aren't what I orignally intended. Also "offramp" and "onramp". Those steret names are things like "Bathurt street offramp". I don't want the name to be "Bathurst street" and the descriptor "offramp". I rather the name be *Bathurst*, and the descriptor be *street offramp*. 

At this point I thought that I could identify all the street descriptors, identify where in the full name the descriptor would be. Then everything before the descriptor would be the name, and everything other would be an optional suffix.

This did not work. For one there was more descriptors than I thought - 192 for the Sydney data test I've been testing with. Including typos like "roaw" and "avenur". Then I realised that there are streets whose name is also a descriptor: Park Lane is common, but there is also Grove Avenue, Terrace Lane and many many more. 

I could have started to do something fancy like "identify the last occurace of the descriptor and everything before is the name and everything after is a suffix", but it just seemed too much. Maintaining a list of descriptors takes time, especially when I'm doing it over a larger area than Sydney.

Intead, I decided to focus on just the main suffixes: North, East, South, West, Offramp, Onramp and exit. If these appear at the end of the full street name I'll mark them as a suffix, otherwise I will stick to the same logic in `get_name2`

In [26]:
def three_name_model(name):
    suffixes = ["north", "east", "south", "west", "offramp", "onramp", "exit"]
    
    split = name.split()
    
    suffix = None
    descriptor = None
    street_name = None
    
    if len(split) == 1:
        street_name = name
    else:
        # If the last word in the name is a recognised suffix
        if len(split) > 2 and split[-1] in suffixes:
            suffix = split[-1]
            split = split[:-1]
        
        if split[0] == 'the':
            street_name = name
            descriptor = split[-1]
        else:
            descriptor = split[-1]
            street_name = " ".join(split[:-1])
     
    return street_name, descriptor, suffix
    
streets["street_name"], streets["street_description"], streets["street_suffix"] = zip(*streets["name"].map(three_name_model))
streets.groupby("street_name")["id"].count().sort_values(ascending=False).head(20)


street_name
park         93
victoria     66
railway      57
william      57
short        53
king         50
albert       50
station      49
george       47
church       47
stanley      44
james        44
charles      43
campbell     42
edward       42
wentworth    40
john         40
elizabeth    39
gordon       39
arthur       39
Name: id, dtype: int64

The street names still appear fine

In [28]:
streets.groupby("street_description")["id"].count().sort_values(ascending=False).head(40)

street_description
street        12860
place          6362
avenue         5924
road           4485
lane           2242
close          1733
crescent       1651
drive           989
way             848
court           672
parade          472
circuit         392
grove           293
terrace         105
boulevard        63
glen             58
motorway         53
highway          32
square           29
row              27
boulevarde       22
walk             22
esplanade        21
gardens          21
bridge           17
parkway          17
mews             16
circle           15
m7               15
rise             14
glade            14
green            12
loop             12
promenade        11
ridge            11
bay              11
link              9
trail             8
tunnel            8
park              7
Name: id, dtype: int64

The descriptors also mostly appear fine. There is an appearance of "M7" which is a major highway which has many onramps, offramps and exits. But as we want to fine the most common street *name*, not so much the descriptor, this isn't a problem. 

In [29]:
streets.groupby("street_suffix")["id"].count().sort_values(ascending=False)

street_suffix
offramp    99
onramp     80
north      55
south      50
west       46
east       44
exit       21
Name: id, dtype: int64

And finally our suffixes. Pleasingly the cardinal directions have similar occurances. It also makes sense that there are more offramps than onramps as highways start somewhere (so don't need an on-ramp) and you don't want more input into your road than output. 

# The most common street name in Sydney


Finally, the most comomn street name in Sydney is "Park", followed by "Vitoria", "Railway", "William" and "Short". Most suburbs in Sydney would have at least one Park, subsequently many also have a Park Road (most often). Even in Sydney CBD there is a Park Road that goes in between Hyde Park. 

Many of the other street names are related to royalty: Victoria, King, Albert and Alfred (Victoria's second son who actually visited Sydney). William and George were also names of Kings, but they were also common first names. 

A few names relate to early Australian coloinal history:
 * __[Wentworth](https://en.wikipedia.org/wiki/William_Wentworth)__, an explorer (also could be his __[father](https://en.wikipedia.org/wiki/D%27Arcy_Wentworth)__, the first paying passenger to come to the colony)
 * __[Hunter](https://en.wikipedia.org/wiki/John_Hunter_(Royal_Navy_officer))__ the second govenor of the colony
 * __[Cook](https://en.wikipedia.org/wiki/James_Cook): Captain Cook claimed the east coast of Australia for the British
 * __[Macquarie](https://en.wikipedia.org/wiki/Lachlan_Macquarie)__: the fifth govenor of the colony
 * __[Mitchell](https://en.wikipedia.org/wiki/Thomas_Mitchell_(explorer))__: an explorer


In [30]:
streets.groupby("street_name")["id"].count().sort_values(ascending=False).head(50)


street_name
park          93
victoria      66
railway       57
william       57
short         53
king          50
albert        50
station       49
george        47
church        47
stanley       44
james         44
charles       43
campbell      42
edward        42
wentworth     40
john          40
elizabeth     39
gordon        39
arthur        39
hunter        38
smith         37
carrington    37
cross         37
boronia       36
mary          36
rose          36
margaret      35
cook          34
first         34
west          34
waratah       34
macquarie     34
mitchell      34
york          33
rawson        33
thomas        33
second        32
wattle        32
western       32
bridge        31
alfred        31
russell       31
pine          31
francis       31
phillip       30
young         30
hill          30
nelson        30
bellevue      30
Name: id, dtype: int64

My original motivation for this was the most common street names for many states in the USA are plant names like Cedar or Oak. I didn't think we did this in Australia - so I set out to find out. At least in Sydney, we see some plant names in the top 50:  __[Boronia](https://en.wikipedia.org/wiki/Boronia)__, https://en.wikipedia.org/wiki/Rose (although I think more likely to be a name), __[Waratah](https://en.wikipedia.org/wiki/Waratah)__ (NSW state emblem), __[Wattle](https://en.wikipedia.org/wiki/Acacia_sensu_lato)__ (National flower of Australia) and Pine. Pine is really interesting as I don't think there are native pine trees in Sydney. Perhaps named after Norfolk pines which were introduced to Sydney early on. 

But, they are no where close to being the most common. Sure, this data is only for Sydney, but I doubt it will change when we look at the whole nation. Flora is diverse in Australia. Boronias and wattles are more or less nation wide, but Waratahs are only in the south east. Introduced pines are nation wide, but are mostly in state forests without many streets. And different states will then add their own native plants to the mix. A state may have a plant name as the most common street name, but I doubt there will be many in the top 10 street names nation wide. 



In [ ]:
# Get names for streets

In [4]:
def get_name(full_name):
    '''Gets the name of the street: full name minus the last word. If the street name is one word it returns
    the street name'''
    
    split_name = full_name.split()
    if len(split_name) == 1:
        return full_name
    else:
        return ' '.join(split_name[:-1])
    
def get_description(full_name):
      
    split_name = full_name.split()
    if len(split_name) == 1:
        return "None"
    else:
        return split_name[-1]

In [5]:
streets["description"] = streets["name"].apply(lambda x: get_description(x) )
streets["street_name"] = streets["name"].apply(lambda x: get_name(x) )

In [6]:
streets.groupby("street_name")["id"].count().sort_values()

street_name
1                     1
mccausland            1
mccleer               1
mccleers              1
mccombe               1
mcconville            1
mccormick             1
mccourtstreet         1
mccowen               1
mccoy                 1
mccrea                1
mccredie              1
mccrossin             1
mccrr                 1
mccubbens             1
mccusker              1
mcdonald's            1
mcdonald's drive      1
mcdonnell             1
mcekhone              1
mcelhone              1
mcencroe              1
mcfadyen              1
mcfall                1
mccarthys             1
mcgann                1
mccarrs creek         1
mcbrian               1
mayberry              1
maybrook              1
                   ... 
waratah              34
margaret             35
mary                 35
hunter               36
rose                 36
boronia              36
carrington           37
wentworth            37
smith                37
cross                37
eliz

In [7]:
the_road = streets[streets["street_name"] == "the"]

colours = ["blue", "red", "green","yellow", "purple", "orange", "pink", "black"]
i = 0
for road in the_road.iterrows():
    print(road[1].geometry)
    for geo in road[1]["geometry"]:
        print(road[1]["geometry"])
        plt.scatter(*zip(*geo.coords[:]), linewidths = 0, color = colours[i%8] )
    i += 1
    #sdfds

AttributeError: 'str' object has no attribute 'coords'

In [8]:
streets.groupby("description")["id"].count().sort_values()

description
(closed)              1
off                   1
outpost               1
oval                  1
palce                 1
palisade              1
paradise              1
parapet               1
parking               1
pass                  1
path                  1
pathway               1
peninsula             1
piazza                1
pickup/dropoff        1
pl                    1
placde                1
place.                1
place]                1
plateau               1
oceania               1
plave                 1
northbound            1
meadows               1
groave                1
groved                1
grovet                1
high                  1
higheay               1
highwater             1
                  ...  
esplanade            21
exit                 21
highway              22
walk                 22
boulevarde           22
row                  27
square               29
east                 44
west                 46
south                50
nort

## Why so many places
I think there is just a lots of new streets with the descriptor place. If I limit it to central Sydney it should be better suspect it will dissapear  after a bit

In [9]:
streets[streets["description"] == "place"].groupby("street_name")["name"].count().sort_values(ascending = False).head()

street_name
finch      10
cox         7
opal        7
banksia     7
viola       6
Name: name, dtype: int64

In [10]:
streets[streets["description"] == "glen"]

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,ref,service,tunnel,type,z_order,description,street_name,geometry_full
1377,arbor glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[22055.0]],arbor glen,[[0]],[[27807793.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,arbor,LINESTRING (150.98536834469 -33.70505703891382...
1658,ascott glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[8960.0]],ascott glen,[[0]],[[16478117.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,ascott,LINESTRING (150.9323994943182 -33.712769982395...
1854,auber glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[52292.0]],auber glen,[[0]],[[173008828.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,auber,LINESTRING (150.7875943218258 -33.808595922788...
2583,barley glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[22504.0]],barley glen,[[0]],[[28394437.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,barley,LINESTRING (150.7327824585842 -33.742793372829...
2862,bax glen,0,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[52309.0], [97139.0]]]",bax glen,"[[[0], [0]]]","[[[173008845.0], [337986682.0]]]","[[[None], [None]]]","[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['residential']]]","[[[3.0], [3.0]]]",glen,bax,MULTILINESTRING ((150.7847962749084 -33.808183...
3167,belbowrie glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[93329.0]],belbowrie glen,[[0]],[[307290023.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,belbowrie,LINESTRING (150.7850279507121 -33.804498764697...
3781,billabong glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[22521.0]],billabong glen,[[0]],[[28394479.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,billabong,LINESTRING (150.7395523541394 -33.740406374443...
3990,blackbird glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[75266.0]],blackbird glen,[[0]],[[203048325.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,blackbird,LINESTRING (150.8089759700882 -33.800403450626...
5234,brolga glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[98209.0]],brolga glen,[[0]],[[353448104.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,brolga,LINESTRING (150.7930448220024 -33.795540857138...
5988,bushy glen,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[86779.0]],bushy glen,[[0]],[[255945636.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],glen,bushy,LINESTRING (150.9975331684038 -33.710285334837...


In [11]:
streets[streets["street_name"] == "glen"]

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,ref,service,tunnel,type,z_order,description,street_name,geometry_full
14935,glen avenue,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[51874.0]],glen avenue,[[0]],[[172961216.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],avenue,glen,LINESTRING (151.2482336292097 -33.914277137294...
14941,glen lane,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[47711.0]],glen lane,[[0]],[[172380264.0]],[[None]],[[None]],[[0]],[['service']],[[0.0]],lane,glen,LINESTRING (151.1800125649092 -33.878236546210...
14942,glen lane,1,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[57377.0]],glen lane,[[0]],[[173671537.0]],[[None]],[[None]],[[0]],[['residential']],[[3.0]],lane,glen,LINESTRING (151.2495879771245 -33.912248800545...
14946,glen place,0,[[[None]]],[[[0]]],[[['highway']]],[<shapely.geometry.linestring.LineString objec...,[[[3443.0]]],glen place,[[[0]]],[[[5757629.0]]],[[[None]]],[[[None]]],[[[0]]],[[['residential']]],[[[3.0]]],place,glen,LINESTRING (150.9498254710159 -33.807759325032...
14947,glen place,1,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[20710.0], [103553.0]]]",glen place,"[[[0], [0]]]","[[[26485126.0], [386727778.0]]]","[[[None], [None]]]","[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['service']]]","[[[3.0], [0.0]]]",place,glen,LINESTRING (150.7726597820273 -34.044273666174...
14948,glen road,0,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[2763.0], [2766.0]]]",glen road,"[[[0], [1]]]","[[[5074135.0], [5074136.0]]]","[[[None], [None]]]","[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['residential']]]","[[[3.0], [3.0]]]",road,glen,MULTILINESTRING ((151.1696940230075 -33.787810...
14949,glen road,1,"[[[None], [None], [None]]]","[[[0], [0], [0]]]","[[['highway'], ['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[10774.0], [10774.0], [26577.0]]]",glen road,"[[[0], [0], [0]]]","[[[22413280.0], [22413280.0], [36184253.0]]]","[[[None], [None], [None]]]","[[[None], [None], [None]]]","[[[0], [0], [0]]]","[[['residential'], ['residential'], ['resident...","[[[3.0], [3.0], [3.0]]]",road,glen,MULTILINESTRING ((151.0696932242816 -33.975245...
14950,glen road,2,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[11611.0], [11611.0]]]",glen road,"[[[0], [0]]]","[[[22698221.0], [22698221.0]]]","[[[None], [None]]]","[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['residential']]]","[[[3.0], [3.0]]]",road,glen,LINESTRING (150.6461569980285 -33.744401776229...
14951,glen road,3,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[48832.0], [48832.0]]]",glen road,"[[[0], [0]]]","[[[172536579.0], [172536579.0]]]","[[[None], [None]]]","[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['residential']]]","[[[3.0], [3.0]]]",road,glen,LINESTRING (151.0202144307532 -33.733277481975...
14952,glen street,0,[[[None]]],[[[0]]],[[['highway']]],[<shapely.geometry.linestring.LineString objec...,[[[6656.0]]],glen street,[[[0]]],[[[13362469.0]]],[[[None]]],[[[None]]],[[[0]]],[[['residential']]],[[[3.0]]],street,glen,LINESTRING (151.2849719299144 -33.776948873890...


# Three name model

Street names are often thought of as being <name> <descriptor> (street, avenue, parade etc.), but this analysis has shown that often you will have variations like "Botany Road East", "M5 Motorway offramp" - like a suffic secriptor

But of course there can be grove street (and terrace street etc.), so really we would need to identify the last occurance of this street descriptor

To remove when reducing streets
* Remove exit from the name - maybe? Its kinda interesting to see how many exits there are

In [12]:
def three_name_model(name):
    descriptors = ["street", "place", "avenue", "road", "lane", "close", "crescent", "drive", "way", "court",
                  "parade", "circuit", "grove", "terrace", "boulevard", "glen", "boulevarde", "highway", "motorway",
                  "esplanade", "mews", "row"]
    
    suffixes = ["north", "east", "south", "west", "offramp", "onramp", "exit"]
    
    split = name.split()
    
    
    suffix = None
    descriptor = None
    street_name = None
    

    if len(split) == 1:
        street_name = name
    else:
        if len(split) > 2 and split[-1] in suffixes:
            suffix = split[-1]
            split = split[:-1]
        
        if split[0] == 'the':
            street_name = " ".join(split)
            descriptor = split[-1]
        else:
            descriptor = split[-1]
            street_name = " ".join(split[:-1])
     
    return street_name, descriptor, suffix
    
    # Find if there is any of the suffixes exist as the last words

In [2]:
streets = streets[streets["name"] != "81-86 courallie avenue homebush west 2140"]

streets["name3"], streets["descriptor3"], streets["suffix3"] = zip(*streets["name"].map(three_name_model))
# streets["name"].apply(three_name_model)

NameError: name 'streets' is not defined

In [1]:
street_rank = streets.groupby("name3")["name"].count().sort_values(ascending = False).reset_index()#.to_csv("popularity.csv")

NameError: name 'streets' is not defined

In [15]:
streets.groupby("descriptor3")["name"].count().sort_values(ascending = False)

descriptor3
street            12860
place              6362
avenue             5924
road               4485
lane               2242
close              1733
crescent           1651
drive               989
way                 848
court               672
parade              472
circuit             392
grove               293
terrace             105
boulevard            63
glen                 58
motorway             53
highway              32
square               29
row                  27
walk                 22
boulevarde           22
esplanade            21
gardens              21
bridge               17
parkway              17
mews                 16
m7                   15
circle               15
glade                14
                  ...  
hill                  1
highwater             1
higheay               1
high                  1
grovet                1
groved                1
northbound            1
off                   1
pocket                1
offramp               1
plav

In [16]:
# m7, northbound, north, south east west, offramp, on ramp

In [17]:
# streets.groupby("description")["id"].count().sort_values(ascending = False).head(50)

In [30]:
streets[streets["name"] == "m7"]

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,...,service,tunnel,type,z_order,description,street_name,geometry_full,name3,descriptor3,suffix3


In [28]:
streets["descriptor3"].unique()

array([None, 'avenue', 'street', 'way', 'road', 'circuit', 'place',
       'close', 'lane', 'row', 'drive', 'parade', 'crescent', 'grove',
       'court', 'terace', 'reserve', 'gardens', 'st', 'off', 'promenade',
       'rise', 'square', 'atreet', 'ave', 'mews', 'glen', 'terrace',
       'closed)', 'circle', 'boulevarde', 'boulevard', 'plave', 'grovet',
       'africa', 'asia', 'europe', 'oceania', 'americas', 'green',
       'garden', 'bypass', 'plaza', 'island', 'link', 'view', 'prade',
       'cvenue', 'esplanade', 'cove', 'bridge', 'parkway', 'offramp',
       'promatary', 'serviceway', 'ramp', 'club', 'ridge', 'walk', 'trail',
       'highway', 'southbound', 'end', 'common', 'bend', 'loop', 'parking',
       'entry', 'path', 'deviation', 'area', 'only', 'station', 'stops',
       'expressway', 'chase', 'park', 'avenuue', 'layover)', 'dtreet',
       'glade', 'quay', 'raiver', 'toe', 'tunnel', 'rd', '6-12',
       'cycleway', 'roaw', 'distributor', 'subway', 'track', 'circuits',
  

In [23]:
streets[streets["street_name"] == "place"]

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,...,service,tunnel,type,z_order,description,street_name,geometry_full,name3,descriptor3,suffix3


In [44]:
streets[streets['suffix3'] == 'north'].head(50)

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,...,service,tunnel,type,z_order,description,street_name,geometry_full,name3,descriptor3,suffix3
3,12th avenue north,0,[['no']],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[79920.0]],12th avenue north,[[0]],[[224697018.0]],...,[[None]],[[0]],[['service']],[[0.0]],north,12th avenue,LINESTRING (151.219416227004 -33.9510610355526...,12th,avenue,north
630,alfred street north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[26158.0]],alfred street north,[[0]],[[34551270.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,alfred street,LINESTRING (151.2124334311083 -33.833945061098...,alfred,street,north
660,alice street north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[11443.0]],alice street north,[[0]],[[22633291.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,alice street,LINESTRING (151.0694890411203 -33.920559457542...,alice,street,north
3347,belmore road north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[60425.0]],belmore road north,[[0]],[[173984873.0]],...,[[None]],[[0]],[['secondary']],[[5.0]],north,belmore road,LINESTRING (151.0562623979126 -33.931823729579...,belmore,road,north
5664,bundarra avenue north,0,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[67932.0], [98611.0]]]",bundarra avenue north,"[[[0], [1]]]","[[[174930370.0], [360957929.0]]]",...,"[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['residential']]]","[[[3.0], [3.0]]]",north,bundarra avenue,MULTILINESTRING ((151.1124402599143 -33.713888...,bundarra,avenue,north
5843,burns road north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[72580.0]],burns road north,[[0]],[[182675895.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,burns road,LINESTRING (151.056340265793 -33.7525366647163...,burns,road,north
8036,clarke street north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[14970.0]],clarke street north,[[0]],[[23296943.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,clarke street,LINESTRING (151.0457483900288 -33.960245338316...,clarke,street,north
8440,college road north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[76230.0]],college road north,[[0]],[[207378158.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,college road,LINESTRING (151.1591084336706 -33.817286531455...,college,road,north
8693,consul road north,0,"[[[None], [None]]]","[[[0], [0]]]","[[['highway'], ['highway']]]",[<shapely.geometry.linestring.LineString objec...,"[[[9790.0], [83501.0]]]",consul road north,"[[[0], [0]]]","[[[19689783.0], [243449228.0]]]",...,"[[[None], [None]]]","[[[0], [0]]]","[[['residential'], ['trunk']]]","[[[3.0], [8.0]]]",north,consul road,LINESTRING (151.2699737714709 -33.757449381985...,consul,road,north
9593,croydon street north,0,[[None]],[[0]],[['highway']],[<shapely.geometry.linestring.LineString objec...,[[11448.0]],croydon street north,[[0]],[[22633309.0]],...,[[None]],[[0]],[['residential']],[[3.0]],north,croydon street,LINESTRING (151.0747170855868 -33.918446463571...,croydon,street,north


In [45]:
streets[streets['name3'] == 'north rooty hill']

,name,Unnamed: 1,access,bridge,class,geometry,id,name.1,oneway,osm_id,...,service,tunnel,type,z_order,description,street_name,geometry_full,name3,descriptor3,suffix3
